- target-folder
    - English_EN
        - MaxiCharger AC
            - Wallbox
                - Installation and Operation Manual
                - Quick Guide
                - Pedestal EP
                    - Quick Guide
            - Compact
                - ...
            - Ultra
                - ...
            - Maintenance Manual
            - Pedestal
                - Quick Guide
            - Pedestal ASP
                - Quick Guide
        - MaxiCharger DC
            - Fast
                - Installation and Operation Manual
                - Service Manual
                - CMS
                    - Quick Gduie
            - Compact
                - ...
        - Energy Management System
            - Operation and Installation Manul
    - German_DE
        - ...

### Target Folder Description
Under folder *"target-folder\English_EN\MaxiCharger AC\Ultra\User Manual"* there are 3 pdf-files and 1 txt-files,<br>
one of the pdf-files has unregular name and should not be identified,<br>
only the pdf-file that edited on 2023-07-06 should be verified.<br>
<br>
The document under folder *"target-folder\English_EN\MaxiCharger DC\Fast\CMS\Quick Guide"* does not exist,<br>
so that this document should be recognized as *missing*<br>
<br>
Under folder *"target-folder\French_FR\MaxiCharger AC\Wallbox\Installation and Operation Manual"* there is one pdf-files named as *"User Mnual"*,<br>
which does not comply with the naming regulation and should be recognized as *abnormal*<br>
<br>
In summary, 16 documents shall be available, but only 13 documents will be found, 1 of those is abnormal.

In [1]:
import os
import re

verif = False
doc_num = 0
fund_num = 0

for root, dirs, files in os.walk(os.path.join('target-folder', 'English_EN')):
    if "arch" in dirs:
        doc_num += 1
        print(root, "...", dirs, "...", files)
        pattern_1 = r".*\\([A-Z][a-zA-Z]+)_(\w{2})\\(MaxiCharger (AC|DC))(.*\\)(.+ (Guide|Manual))$"
        pattern_2 = r".*\\([A-Z][a-zA-Z]+)_(\w{2})\\(Energy Management System)(.*\\)(.+ (Guide|Manual))$"
        match_1 = re.match(pattern_1, root)
        match_2 = re.match(pattern_2, root)
        if match_1:
            category = match_1.group(3)
            item = match_1.group(5).replace("\\", "").strip()
            document = match_1.group(6).replace("\\", "") 
        elif match_2:
            category = match_2.group(3)
            item = ''
            document = match_2.group(5).replace("\\", "") 
        else:
            print("Document path '%s' not correct." % (root))
        if len(files)!=0:
            fund_num += 1
            #print("Document {0} {1} {2} exists.".format(category, item, document))
        else:
            print("Document {0} {1} {2} is missing.".format(category, item, document))
    else:
        pass

print("%d documents shall be available." % (doc_num))
print("%d documents has been found." % (fund_num))

target-folder\English_EN\MaxiCharger AC\Compact\User Manual ... ['arch'] ... ['MaxiCharger AC Compact - User Manual_20230302_EN.pdf']
target-folder\English_EN\MaxiCharger AC\Compact\Quick Guide ... ['arch'] ... ['MaxiCharger AC Compact - Quick Guide_20230215_EN.pdf']
target-folder\English_EN\MaxiCharger AC\Ultra\User Manual ... ['arch'] ... ['MaxiCharger AC Ultra - User Manual EU_20230712_EN.txt', 'MiniCharger AC Ultra - User Manual EU_20230505_EN.pdf', 'Some - Random File_20220303_EN.pdf', 'MaxiCharger AC Ultra - User Manual EU_EN.pdf', 'MaxiCharger AC Ultra - User Manual_EN.pdf']
target-folder\English_EN\MaxiCharger AC\Ultra\Quick Guide ... ['arch'] ... ['MaxiCharger AC Ultra - Quick Guide_20230330_EN.pdf']
target-folder\English_EN\MaxiCharger AC\Wallbox\Quick Guide ... ['arch'] ... ['MaxiCharger AC Wallbox - Quick Guide_20230523_EN.pdf']
target-folder\English_EN\MaxiCharger AC\Wallbox\Pedestal EP\Quick Guide ... ['arch'] ... ['MaxiCharger AC Wallbox Pedestal EP  -  Quick Guide_20230

#### Function 1: path_matcher()
The function *path_matcher()* analizes the path as argument, extract the information that the name of document under this path **MUST** contain.

In [2]:
import re

#root = 'target-folder\\English_EN\\MaxiCharger AC\\Wallbox\\Pedestal EP\\Quick Guide'
root = 'target-folder\\English_EN\\MaxiCharger AC\\Wallbox\\Installation and Operation Manual'
#root = 'target-folder\\English_EN\\MaxiCharger AC\\Wallbox\\Pedestal EP\\'
#root = 'target-folder\\English_EN\\Energy Management System\\Operation and Installation Manual'
#root = 'target-folder\\English_EN\\MaxiCharger AC\\Maintenance Manual'

def path_extractor(root:str) -> tuple[str, str, str]:
    pattern_1 = r".*\\([A-Z][a-zA-Z]+)_(\w{2})\\(MaxiCharger (AC|DC))(.*)(\\.+ (Guide|Manual))$"
    pattern_2 = r".*\\([A-Z][a-zA-Z]+)_(\w{2})\\(Energy Management System)(.*)(\\.+ (Guide|Manual))$"
    match_1 = re.match(pattern_1, root)
    match_2 = re.match(pattern_2, root)
    if match_1:
        category = match_1.group(3)
        object = match_1.group(5).replace("\\", " ").strip()
        document = match_1.group(6).replace("\\", "")     
    elif match_2:
        category = match_2.group(3)
        object = 'n/a'
        document = match_2.group(5).replace("\\", "")  
    else:
        print("Path '%s' is not correct." % (root))
        category = 'n/a'
        object = 'n/a'
        document = 'n/a'
    return (category, object, document)


####################################################################################################################
# Test
t = path_extractor(root)
print(t)

('MaxiCharger AC', 'Wallbox', 'Installation and Operation Manual')


#### Function 2: file_matcher()
The function *file_matcher* analizes the *list of files* as argument under given path. Collect name and release date of pdf-file in a *dictionary*, who's name complies the naming regulation. The pdf-file that edited recently  

In [3]:
import os
import re

from datetime import datetime


files = ['MaxiCharger AC Ultra - User Manual EU_EN.pdf', 
         'MaxiCharger AC Ultra - User Manual EU_20230712_EN.txt',
         'MaxiCharger AC Ultra - User Manual_EN.pdf', 
         'MiniCharger AC Ultra - User Manual EU_20230505_EN.pdf',
         'Some - Random File_20220303_EN.pdf']
root = 'target-folder\\English_EN\\MaxiCharger AC\\Ultra\\User Manual'
path_info = ('MaxiCharger AC', 'Ultra', 'User Manual')


def file_matcher(root:str, path_info:tuple[str, str, str], files:list) -> tuple[str, str, int]:
    # initial dict of all existing pdf files
    pdf_files = {}
    result = 0
    bad_result = 0
    # get formal file name
    p1, p2, p3 = path_info
    file_name = (p1 + ' ' + p2 if len(p2)>0 else p1) + ' - ' + p3
    #pattern_part = r"%s %s - %s(?:.*)?.*.pdf$" % path_info
    #pattern_full = r"%s %s - %s(?:.*)?_(\d{4})(\d{2})(\d{2})_(\w{2}).pdf$" % path_info
    #pattern_pdf = r".*.pdf$"
    pattern_part = file_name + '(?:.*)?.*.pdf$'
    pattern_full = file_name + '(?:.*)?_(\d{4})(\d{2})(\d{2})_(\w{2}).pdf$'
    # if appropriate pdf file exists under the 
    for file in files:
        # fuzzy match
        match_part = re.match(pattern_part, file)    # re.IGNORECASE
        if match_part:
            match_full = re.match(pattern_full, file)    # re.IGNORECASE
            if match_full:
                release = f"{match_full.group(1)}-{match_full.group(2)}-{match_full.group(3)}"
            else:
                release = datetime.fromtimestamp(os.path.getmtime(os.path.join(root, file))).strftime(r"%Y-%m-%d")
            pdf_files.setdefault(file, release)
        else:
            #match_pdf = re.match(pattern_pdf, file)
            if file.endswith('.pdf'):
                if all(element.lower() in file.lower() for element in path_info):
                    bad_result = 2
                    print(f"Unregular named file: {file}")
                else:
                    bad_result = 3
                    print(f"Unknown pdf file: {file}")
            else:
                
                print(f"Non-pdf file: {file}")
            pass
    try:
        # returns full file name and its release date
        latest_file, latest_release = max(pdf_files.items(), key=lambda item: item[1])
        result = 1 if len(pdf_files) >= 0 else bad_result
        print(f"Latest file {latest_file} edited on {latest_release} verified.")
    except ValueError:
        latest_file = ''
        latest_release = ''
        result = 0
        print(f"Document {file_name} is missing.")
    return (latest_file, latest_release, result)


####################################################################################################################
# Test
m = file_matcher (root, path_info, files)
lf, lr, r = m
print(m)
print(lf, lr, r)

Non-pdf file: MaxiCharger AC Ultra - User Manual EU_20230712_EN.txt
Unknown pdf file: MiniCharger AC Ultra - User Manual EU_20230505_EN.pdf
Unknown pdf file: Some - Random File_20220303_EN.pdf
Latest file MaxiCharger AC Ultra - User Manual_EN.pdf edited on 2023-07-06 verified.
('MaxiCharger AC Ultra - User Manual_EN.pdf', '2023-07-06', 1)
MaxiCharger AC Ultra - User Manual_EN.pdf 2023-07-06 1


In [4]:
d = {'MaxiCharger AC Ultra - User Manual EU_EN.pdf': '2023-06-23', 'MaxiCharger AC Ultra - User Manual_20230706_EN.pdf': '2023-07-06'}
max_key, max_value = max(d.items(), key=lambda item: item[1])
print(max_key, max_value)

MaxiCharger AC Ultra - User Manual_20230706_EN.pdf 2023-07-06


In [5]:
from datetime import datetime

current_date = datetime.now().strftime(r"%Y-%m-%d")
print(current_date)

2023-08-15


#### Function 3: file_detector()
Combination of function 1 and function 2.

In [7]:
import os
import re

from datetime import datetime


####################################################################################################################
# Test Cases

test_cases = {
    'case 0': {
        'root': 'target-folder\\English_EN\\MaxiCharger DC\\Fast\\CMS\\Quick Guide',
        'dirs': ['arch'],
        'files': [],
        'result': ('MaxiCharger DC', 'Fast CMS', 'Quick Guide', '', 0)
    },
    'case_1': {
        'root': 'target-folder\\English_EN\\MaxiCharger AC\\Ultra\\User Manual',
        'dirs': ['arch'],
        'files': ['MaxiCharger AC Ultra - User Manual EU_EN.pdf', 
                  'MaxiCharger AC Ultra - User Manual EU_20230712_EN.txt',
                  'MaxiCharger AC Ultra - User Manual_EN.pdf', 
                  'MiniCharger AC Ultra - User Manual EU_20230505_EN.pdf', 
                  'Some - Random File_20220303_EN.pdf'],
        'result': ('MaxiCharger AC', 'Ultra', 'User Manual', '2023-07-06', 1)
    },
    'case_2': {
        'root': 'target-folder\\English_EN\\Energy Management System\\Operation and Installation Manual',
        'dirs': ['arch'],
        'files': ['Energy Management System - Operation and Installation Manual EU_20230525_EN.pdf'],
        'result': ('Energy Management System', '', 'Operation and Installation Manual', '2023-05-25', 1)
    },
    'case_3': {
        'root': 'target-folder\\English_EN\\MaxiCharger AC\\Maintenance Manual',
        'dirs': ['arch'],
        'files': ['MaxiCharger AC - Maintenance Manual EU_20220224_EN.pdf'],
        'result': ('MaxiCharger AC', '', 'Maintenance Manual', '2022-02-24', 1)
    },
    'case_4': {
        'root': 'target-folder\\English_EN\\MaxiCharger AC\\Wallbox\\Pedestal EP\\Quick Guide',
        'dirs': ['arch'],
        'files': ['MaxiCharger AC Wallbox Pedestal EP  -  Quick Guide_20230512_EN.pdf'],
        'result': ('MaxiCharger AC', 'Wallbox Pedestal EP', 'Quick Guide', '', 2)
    },
    'case_5': {
        'root': 'target-folder\\English_EN\\MaxiCharger AC\\Pedestal ASP\\Quick Guide',
        'dirs': ['arch'],
        'files': ['Some unknown - Guide_20240404_EN.pdf'],
        'result': ('MaxiCharger AC', 'Pedestal ASP', 'Quick Guide', '', 3)
    },
}


####################################################################################################################
# Function Build

def file_detector (root:str, files:list) -> tuple[str, str, str, str, bool]:
    ###########################
    # Part 1: Path Extraction #
    ###########################
    pattern_1 = r".*\\([A-Z][a-zA-Z]+)_(\w{2})\\(MaxiCharger (AC|DC))(.*)(\\.+ (Guide|Manual))$"
    pattern_2 = r".*\\([A-Z][a-zA-Z]+)_(\w{2})\\(Energy Management System)(.*)(\\.+ (Guide|Manual))$"
    match_1 = re.match(pattern_1, root)
    match_2 = re.match(pattern_2, root)
    try:
        if match_1:
            category = match_1.group(3)
            item = match_1.group(5).replace("\\", " ").strip()
            document = match_1.group(6).replace("\\", "")     
        elif match_2:
            category = match_2.group(3)
            item = ''
            document = match_2.group(5).replace("\\", "")  
    except AttributeError:
        print("Path '%s' is not a proper document path." % (root))
        category = 'null'
        item = 'null'
        document = 'null'
    path_info = (category, item, document)
    #########################
    # Part 2: File Matching #
    #########################
    pdf_files = {}
    result = 0
    bad_result = 0
    # get formal file name
    formal_name = (category + ' ' + item if len(item)>0 else category) + ' - ' + document
    pattern_without_time = formal_name + '(?:.*)?.*.pdf$'
    pattern_with_time = formal_name + '(?:.*)?_(\d{4})(\d{2})(\d{2})_(\w{2}).pdf$'
    # if appropriate pdf file exists under the 
    for file in files:
        # fuzzy match
        match_part = re.match(pattern_without_time, file, re.IGNORECASE)
        if match_part:
            # accurate match
            match_full = re.match(pattern_with_time, file, re.IGNORECASE)
            if match_full:
                release = f"{match_full.group(1)}-{match_full.group(2)}-{match_full.group(3)}"
                pass
            else:
                release = datetime.fromtimestamp(os.path.getmtime(os.path.join(root, file))).strftime(r"%Y-%m-%d")
                pass
            pdf_files.setdefault(file, release)
        else:
            if file.endswith('.pdf'):
                # pdf-file name with currect path information but not match
                if all(element.lower() in file.lower() for element in path_info):
                    bad_result = 2
                # unknown pdf-file name
                else:
                    bad_result = 3
            else:
                # no pdf-file
                bad_result = 0
    #############################
    # Part 3: Result Evaluation #
    #############################
    debug = {
        0: "Missing document for {0}.",
        1: "Verified document for {0} edited on {1}.",
        2: 'Unregular named document for {0}.',
        3: 'Unknown pdf-file for {0}.'
    }
    # returns detection result
    try:
        _, release = max(pdf_files.items(), key=lambda item: item[1])
        result = 1
        print(debug[result].format(formal_name, release))
    except ValueError:
        release = ''
        result = bad_result
        print(debug[result].format(formal_name))
    return category, item, document, release, result


####################################################################################################################
# Function Test

for case, content in test_cases.items():
    test_result = file_detector(content['root'], content['files'])
    if content['result'] == test_result:
        print(f"{case} passed.")
    else:
        print(f"{case} failed.")



Missing document for MaxiCharger DC Fast CMS - Quick Guide.
case 0 passed.
Verified document for MaxiCharger AC Ultra - User Manual edited on 2023-07-06.
case_1 passed.
Verified document for Energy Management System - Operation and Installation Manual edited on 2023-05-25.
case_2 passed.
Verified document for MaxiCharger AC - Maintenance Manual edited on 2022-02-24.
case_3 passed.
Unregular named document for MaxiCharger AC Wallbox Pedestal EP - Quick Guide.
case_4 passed.
Unknown pdf-file for MaxiCharger AC Pedestal ASP - Quick Guide.
case_5 passed.


#### Function 4: file_recorder()
The function file extractor get the edit and release times, also the language code of the selected pdf file.<br>
This function refers to function 3: *file_detector*

In [8]:
import os
import re
import csv

from datetime import datetime


target_folder = "target-folder"
# ['English_EN', 'French_FR', 'German_DE']
language_folders = [folder for folder in os.listdir(target_folder) if os.path.isdir(os.path.join(target_folder, folder))]


def file_recorder (folders: list):
    # iterate over  all language folders
    for folder in folders:
        # extract language information
        try:
            match = re.match(r"([A-Z][a-zA-Z]+)_(\w{2})", folder, re.IGNORECASE)
            language = match.group(1)  # English
            language_code = match.group(2)  # EN
        except AttributeError:
            print("Language folder abnormal")
            continue
        # initial basic information
        doc_num = 0
        fund_num = 0
        
        now = datetime.now().strftime(r"%Y%m%d%H%M%S")
        ########################
        # Part 1: Log Cleaning #
        ########################
        folder_path = os.path.join(target_folder, folder)
        # list of all csv files
        logs = [file for file in os.listdir(folder_path) if file.endswith('.csv')]
        if len(logs) != 0:
            # pair of name and edit time in tuple
            #log_with_time = [(log, os.path.getmtime(os.path.join(folder_path, log))) for log in logs]
            # arrange in ascending order
            #log_with_time.sort(key=lambda x: x[1])    # for log, _ in log_with_time[:-1]:
            for log in logs:
                log_path = os.path.join(folder_path, log)
                os.remove(log_path)
                print(f"Existing Log cleaned: {log_path}")
        else:
            print("No existing log.")
        ##########################
        # Part 2: File Recording #
        ##########################
        csv_name = f"{language_code}_log_{now}.csv"
        csv_path = os.path.join(target_folder, folder, csv_name)
        with open(csv_path, 'w', newline='') as csv_file:
            csv_writer = csv.writer(csv_file)
            csv_writer.writerow(['language', 'category', 'item', 'document', 'release', 'status'])
            # iterate over all documents in each folder
            for root, dirs, files in os.walk(os.path.join('target-folder', folder)):
                # only folder with "arch" sub-folder is target
                if "arch" in dirs:
                    doc_num += 1
                    try:
                        # process the document folder
                        category, item, document, release, result = file_detector(root, files)
                        if result == 1:
                            fund_num += int(result)
                            status = "verified"
                        elif result == 2:
                            status = "abnormal"
                        else:
                            status = "missing"
                    except NameError as n_error:
                        print(f"An error occurred: {n_error}")
                else:
                    continue
                csv_writer.writerow([language_code, category, item, document, release, status])
        print(f"{doc_num} {language} documents shall be available.")
        print(f"{fund_num} {language} documents has been found.")
        print(f"New log saved: {csv_path}")
    return


####################################################################################################################
# Test
file_recorder(language_folders)


Existing Log cleaned: target-folder\English_EN\EN_log_20230915010015.csv
Verified document for MaxiCharger AC Compact - User Manual edited on 2023-03-02.
Verified document for MaxiCharger AC Compact - Quick Guide edited on 2023-02-15.
Verified document for MaxiCharger AC Ultra - User Manual edited on 2023-07-06.
Verified document for MaxiCharger AC Ultra - Quick Guide edited on 2023-03-30.
Verified document for MaxiCharger AC Wallbox - Quick Guide edited on 2023-05-23.
Unregular named document for MaxiCharger AC Wallbox Pedestal EP - Quick Guide.
Verified document for MaxiCharger AC Wallbox - Installation and Operation Manual edited on 2023-07-17.
Verified document for MaxiCharger AC Wallbox Pedestal - Quick Guide edited on 2023-02-23.
Verified document for MaxiCharger AC - Maintenance Manual edited on 2022-02-24.
Unknown pdf-file for MaxiCharger AC Pedestal ASP - Quick Guide.
Verified document for MaxiCharger DC Compact - Installation and Operation Manual edited on 2023-05-05.
Verifie

#### Function 5: log_analyzer()
The function file

In [5]:
import os
import re
import sys
import pandas as pd


target_folder = "test-folder"
md_path = os.path.join(target_folder, 'README.md')
# ['English_EN', 'German_DE']
lang_folders = [folder for folder in os.listdir(target_folder) if os.path.isdir(os.path.join(target_folder, folder))]

# get status from 'English_EN' first
en_folder_path = os.path.join(target_folder, "English_EN")
en_log_name = [file for file in os.listdir(en_folder_path) if file.endswith('.csv')][0]

# try to get english log
#test-folder\English_EN\EN_log_20230816002603.csv
try:
    en_log_csv_path = os.path.join(en_folder_path, en_log_name)
    log_df = pd.read_csv(en_log_csv_path)
except IndexError:
    print(f"English log is missing, process shutted down.")
    #return
    sys.exit()

# clean the columns 'language'
log_df.drop(['language', 'status'], axis=1, inplace=True)
log_df.rename(columns={'release': 'EN'}, inplace=True)

##############################################################################

# analyze languages except english
lang_folders.remove("English_EN")

# check other language folders and save in dataframe
for folder in lang_folders:
    # get language code
    try:
        language_code = re.match(r"([A-Z][a-zA-Z]+)_(\w{2})", folder, re.IGNORECASE).group(2)  # EN
    except AttributeError:
        print(f"Folder {folder} unregular.")
    # get path of language folder
    folder_path = os.path.join(target_folder, folder)    # test-folder\German_DE
    # get csv log as list, should be length 1
    logs = [file for file in os.listdir(folder_path) if file.endswith('.csv')]    # ['DE_log_20230816002603.csv']
    # get path of csv log
    try:
        log_csv_path = os.path.join(target_folder, folder, logs[0])    # test-folder\German_DE\DE_log_20230816002603.csv
    except IndexError:
        print(f"{folder} log is missing.\n\n")
        continue





#print(log_df)
print(log_csv_path)

French_FR log is missing.


test-folder\German_DE\DE_log_20230816002603.csv


In [3]:
import re
#match = re.match(r"([A-Z][a-zA-Z]+)_(\w{2})", 'English_EN', re.IGNORECASE)
try:
    #language = match.group(1)  # English
    language_code = re.match(r"([A-Z][a-zA-Z]+)_(\w{2})", 'English_EN', re.IGNORECASE).group(2)  # EN
except AttributeError:
    print("Language folder abnormal")

language_code

'EN'

#### Function 6: md_writer()
The function file extractor get the edit and release times, also the language code of the selected pdf file.

In [12]:
import os
import sys
import pandas as pd

from datetime import datetime


target_folder = "test-folder"
md_path = os.path.join(target_folder, 'README.md')
# ['English_EN', 'German_DE']
lang_folders = [folder for folder in os.listdir(target_folder) if os.path.isdir(os.path.join(target_folder, folder))]


today = datetime.now().strftime(r"%Y-%m-%d")
leadin_1 = (
    "# Overview of Document Translations\n"
    f"last update: {today}\n"
    "## Status of English Documents\n"
)
header = (
    "| language | category | item | document | update | status |\n"
    "| --- | --- | --- | --- | --- | --- |\n"
)
divider = (
    "\n"
    "---\n"
    "\n"
)
leadin_vendor_trans = (
    "## Languages that translated via Vendor:\n"
    "Danish (DA), Dutch (NL), French (FR), German (DE), Italian (IT), Polish (PL), Portuguese (PT), Spanish (ES), Swedish (SV), Turkish (TR)<br>\n"
)
leadin_google_trans = (
    "## Languages that translated via Google:\n"
    "Czech (CZ), Finnish (FI), Icelandic (IS), Norwegian (NO), Romanian (RO), Russian (RU)<br>\n"
)


with open(md_path, "w", newline='') as md_file:
    md_file.write(leadin_1)
    # get status from 'English_EN' first
    en_folder_path = os.path.join(target_folder, "English_EN")
    en_log_name = [file for file in os.listdir(en_folder_path) if file.endswith('.csv')][0]
    # test-folder\English_EN\EN_log_20230816002603.csv
    try:
        en_log_csv_path = os.path.join(en_folder_path, en_log_name)
    except IndexError:
        md_file.write(f"English_EN log is missing, process broke off.\n\n")
        #return
        sys.exit()
    # check english documents
    with open(en_log_csv_path, "r") as en_log:
        # extract all lines as strings in single list
        lines = en_log.readlines()
        md_file.write(header)

        
        # get lines except the title
        for line in lines[1:]:
            # split elements from a single line as a list
            # ['EN', 'MaxiCharger AC', 'Compact', 'User Manual', '2023-03-02', 'verified']
            values = line.strip().split(",")
            md_file.write("| " + " | ".join(values) + " |\n")
        # end of current table
        md_file.write(f"English_EN log is checked.\n\n")
        md_file.write(divider)


        md_file.write(leadin_vendor_trans)
        md_file.write(leadin_google_trans)

        # save as dataframe in cache
        en_log_df = pd.read_csv(en_log_csv_path)
        # analyze languages except english
        lang_folders.remove("English_EN")

        # check other language folders and save in dataframe
        for lang_folder in lang_folders:
            # test-folder\German_DE
            folder_path = os.path.join(target_folder, lang_folder)
            # ['DE_log_20230816002603.csv'], should be only 1
            logs = [file for file in os.listdir(folder_path) if file.endswith('.csv')]
            # test-folder\German_DE\DE_log_20230816002603.csv
            try:
                log_csv_path = os.path.join(target_folder, lang_folder, logs[0])
            except IndexError:
                md_file.write(f"{lang_folder} log is missing.\n\n")
                continue
        #print(os.path.exists(log_csv_path))
        # check english documents
        with open(log_csv_path, "r") as log:
            # extract all lines as strings in single list
            lines = log.readlines()
            md_file.write(header)
            # get lines except the title
            for line in lines[1:]:
                # split elements from a single line as a list
                # ['DE', 'MaxiCharger AC', 'Wallbox', 'Installation and Operation Manual', , 'missing']
                values = line.strip().split(",")
                md_file.write("| " + " | ".join(values) + " |\n")
            # end of current table
            md_file.write(f"{lang_folder} log is checked.\n\n")
            md_file.write(divider)
    


In [10]:
import os

log_fund = False
logs = [file for file in os.listdir('test-folder\French_FR') if file.endswith('.csv')]
if len(logs)>0:
    fr_log_csv_path = os.path.join(target_folder, lang_folder, logs[0])
    log_fund = True
else:
    log_fund = False
    #continue
print(logs)

[]


In [ ]:
import os

# 原始目录和目标目录
source_dir = "./test-folder/English_EN\MaxiCharger AC/Wallbox/User Manual"
target_dir = "./test-folder/English_EN\MaxiCharger AC/Wallbox/User Manual"

# 遍历源目录下的所有文件
for filename in os.listdir(source_dir):
    if filename.endswith(".pdf"):
        # 构建源文件路径和目标文件路径
        source_file = os.path.join(source_dir, filename)
        target_file = os.path.join(target_dir, filename)
        
        # 将目标文件路径中的文件名替换为指定的 filename
        target_file = os.path.join(target_dir, filename.replace(".pdf", "_" + filename))
        
        # 重命名文件
        os.rename(source_file, target_file)

In [1]:
import pandas as pd

# 创建一个示例 DataFrame
data = {'Name': ['Alice', 'Bob', 'Alice', 'Charlie', 'Bob'],
        'Age': [25, 30, 25, 22, 30],
        'City': ['New York', 'San Francisco', 'New York', 'Los Angeles', 'San Francisco']}
df = pd.DataFrame(data)

# 按姓名和年龄排序
df = df.sort_values(by=['Name', 'Age'])

# 初始化 HTML 表格字符串
html_table = '<table border="1">\n'

# 遍历 DataFrame，合并相同单元格并生成 HTML 表格内容
prev_row = None
for index, row in df.iterrows():
    if prev_row is None:
        prev_row = row
        rowspan = 1
    elif row['Name'] == prev_row['Name']:
        rowspan += 1
        continue
    else:
        html_table += f'<tr><td rowspan="{rowspan}">{prev_row["Name"]}</td><td>{prev_row["Age"]}</td><td>{prev_row["City"]}</td></tr>\n'
        rowspan = 1
        prev_row = row

html_table += f'<tr><td rowspan="{rowspan}">{prev_row["Name"]}</td><td>{prev_row["Age"]}</td><td>{prev_row["City"]}</td></tr>\n'
html_table += '</table>'

# 将 HTML 表格写入文件或进行其他操作
with open('output.html', 'w') as f:
    f.write(html_table)

In [5]:
data = [
    {'Category': 'MaxiCharger AC', 'Item': 'Wallbox', 'Document': 'Quick Guide', 'Released': 'yyyy-mm-dd'},
    {'Category': 'MaxiCharger AC', 'Item': 'Wallbox', 'Document': 'User Manual', 'Released': 'yyyy-mm-dd'},
    {'Category': 'MaxiCharger AC', 'Item': 'Wallbox Pedestal', 'Document': 'Quick Guide', 'Released': 'yyyy-mm-dd'},
    {'Category': 'MaxiCharger AC', 'Item': '', 'Document': 'Maintenance Manual', 'Released': 'yyyy-mm-dd'},
    {'Category': 'MaxiCharger DC', 'Item': 'Fast', 'Document': 'Installation Manual', 'Released': 'yyyy-mm-dd'},
    {'Category': 'MaxiCharger DC', 'Item': 'Fast', 'Document': 'Service Manual', 'Released': 'yyyy-mm-dd'},
    {'Category': 'MaxiCharger DC', 'Item': 'Compact', 'Document': 'Quick Guide', 'Released': 'yyyy-mm-dd'},
    {'Category': 'MaxiCharger DC', 'Item': 'Compact', 'Document': 'Service Manual', 'Released': 'yyyy-mm-dd'},
    {'Category': 'Energy Management System', 'Item': '', 'Document': 'Operation Manual', 'Released': 'yyyy-mm-dd'}
    # ...
    # 添加更多数据行
    # ...
]

# 定义 HTML 表格的开始
html_table = '''
<h4>Overview of released English documents:</h4>
<table border="1">
<tr>
    <th>Category</th>
    <th>Item</th>
    <th>Document</th>
    <th>Released</th>
</tr>
'''

# 定义前一个分类和项目
prev_cat = None
prev_item = None

# 遍历数据并生成 HTML 表格行
for row in data:
    cat = row['Category']
    item = row['Item']
    doc = row['Document']
    released = row['Released']
    
    if cat != prev_cat:
        html_table += f'<tr><th rowspan="{data.count(row)}" class="cat">{cat}</th><td class="ite">{item}</td><td class="doc">{doc}</td><td>{released}</td></tr>\n'
    elif item != prev_item:
        html_table += f'<tr><td class="ite">{item}</td><td class="doc">{doc}</td><td>{released}</td></tr>\n'
    
    prev_cat = cat
    prev_item = item

# 完成 HTML 表格
html_table += '</table>'

# 打印生成的 HTML 表格
print(html_table)


<h4>Overview of released English documents:</h4>
<table border="1">
<tr>
    <th>Category</th>
    <th>Item</th>
    <th>Document</th>
    <th>Released</th>
</tr>
<tr><th rowspan="1" class="cat">MaxiCharger AC</th><td class="ite">Wallbox</td><td class="doc">Quick Guide</td><td>yyyy-mm-dd</td></tr>
<tr><td class="ite">Wallbox Pedestal</td><td class="doc">Quick Guide</td><td>yyyy-mm-dd</td></tr>
<tr><td class="ite"></td><td class="doc">Maintenance Manual</td><td>yyyy-mm-dd</td></tr>
<tr><th rowspan="1" class="cat">MaxiCharger DC</th><td class="ite">Fast</td><td class="doc">Installation Manual</td><td>yyyy-mm-dd</td></tr>
<tr><td class="ite">Compact</td><td class="doc">Quick Guide</td><td>yyyy-mm-dd</td></tr>
<tr><th rowspan="1" class="cat">Energy Management System</th><td class="ite"></td><td class="doc">Operation Manual</td><td>yyyy-mm-dd</td></tr>
</table>
